<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания № 4


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----


Вариант задания №9 — Описание и реализация

Задача:
Создать базовый класс Order в C#, который будет представлять информацию о
заказах товаров или услуг. На основе этого класса разработать 2-3 производных
класса, демонстрирующих принципы наследования и полиморфизма. В каждом из
классов должны быть реализованы новые атрибуты и методы, а также
переопределены некоторые методы базового класса для демонстрации
полиморфизма.

Требования к базовому классу Order:
• Атрибуты: ID заказа (OrderId), Дата создания (CreationDate), Сумма заказа (TotalAmount).
• Методы:
  o CalculateTotal(): метод для расчета общей суммы заказа.
  o AddItem(Item item): метод для добавления элемента в заказ.
  o RemoveItem(Item item): метод для удаления элемента из заказа.

Требования к производным классам:
1. ОнлайнЗаказ (OnlineOrder): Должен содержать дополнительные атрибуты,
   такие как Email клиента (CustomerEmail). Метод AddItem() должен быть
   переопределен для добавления информации о способе доставки при
   добавлении элемента.
2. ФизическийЗаказ (PhysicalOrder): Должен содержать дополнительные
   атрибуты, такие как Адрес доставки (DeliveryAddress).
   Метод RemoveItem() должен быть переопределен для добавления
   информации о возврате товара при удалении элемента.
3. СпециализированныйЗаказ (SpecializedOrder): Должен содержать дополнительные
   атрибуты, такие как Специальные условия (SpecialConditions). Метод CalculateTotal()
   должен быть переопределен для учета специальных условий при расчете общей суммы
   заказа.



<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [1]:
using System;
using System.Collections.Generic;
using System.Linq;

public class Item
{
    public string Name { get; set; }
    public decimal Price { get; set; }
    public int Quantity { get; set; }

    public Item(string name, decimal price, int quantity)
    {
        Name = name;
        Price = price;
        Quantity = quantity;
    }

    public decimal Total => Price * Quantity;
}

public class Order
{
    protected List<Item> items = new();
    public IReadOnlyList<Item> Items => items.AsReadOnly();

    public int OrderId { get; set; }
    public DateTime CreationDate { get; set; }
    public decimal TotalAmount { get; protected set; }

    public Order(int orderId)
    {
        OrderId = orderId;
        CreationDate = DateTime.Now;
    }

    public virtual void AddItem(Item item)
    {
        if (item == null) throw new ArgumentNullException(nameof(item));
        items.Add(item);
        CalculateTotal();
    }

    public virtual void RemoveItem(Item item)
    {
        if (item == null) throw new ArgumentNullException(nameof(item));
        items.Remove(item);
        CalculateTotal();
    }

    public virtual void CalculateTotal()
    {
        TotalAmount = items.Sum(i => i.Price * i.Quantity);
    }

    public virtual void DisplayInfo()
    {
        Console.WriteLine($"Заказ #{OrderId} от {CreationDate}");
        foreach (var item in items)
        {
            Console.WriteLine($"- {item.Name} x{item.Quantity} = {item.Price * item.Quantity}₽");
        }

        Console.WriteLine($"Итого: {TotalAmount}₽");
    }
}

public class OnlineOrder : Order
{
    public string CustomerEmail { get; set; }
    public string DeliveryMethod { get; set; }

    public OnlineOrder(int orderId, string email, string method)
        : base(orderId)
    {
        CustomerEmail = email;
        DeliveryMethod = method;
    }

    public override void AddItem(Item item)
    {
        base.AddItem(item);
        Console.WriteLine($"Товар {item.Name} добавлен (доставка: {DeliveryMethod})");
    }

    public override void DisplayInfo()
    {
        Console.WriteLine($"[Онлайн-заказ] Email: {CustomerEmail}");
        base.DisplayInfo();
    }
}

public class PhysicalOrder : Order
{
    public string DeliveryAddress { get; set; }

    public PhysicalOrder(int orderId, string address)
        : base(orderId)
    {
        DeliveryAddress = address;
    }

    public override void RemoveItem(Item item)
    {
        base.RemoveItem(item);
        Console.WriteLine($"Товар {item.Name} удалён. Возврат оформлен по адресу: {DeliveryAddress}");
    }

    public override void DisplayInfo()
    {
        Console.WriteLine($"[Физический заказ] Адрес: {DeliveryAddress}");
        base.DisplayInfo();
    }
}

public class SpecializedOrder : Order
{
    public string SpecialConditions { get; set; }
    public decimal DiscountPercent { get; set; }

    public SpecializedOrder(int orderId, string conditions, decimal discount)
        : base(orderId)
    {
        SpecialConditions = conditions;
        DiscountPercent = discount;
    }

    public override void CalculateTotal()
    {
        base.CalculateTotal();

        if (DiscountPercent != 0m)
        {
            TotalAmount -= TotalAmount * (DiscountPercent / 100m);
        }

        if (!string.IsNullOrEmpty(SpecialConditions) &&
            SpecialConditions.Contains("fragile", StringComparison.OrdinalIgnoreCase))
        {
            TotalAmount = Math.Round(TotalAmount * 1.05m, 2);
        }
    }

    public override void DisplayInfo()
    {
        Console.WriteLine($"[Спецзаказ] Условия: {SpecialConditions}, Скидка: {DiscountPercent}%");
        base.DisplayInfo();
    }
}

// Демонстрация
var onlineOrder = new OnlineOrder(1, "user@example.com", "Курьер");
onlineOrder.AddItem(new Item("Ноутбук", 80000m, 1));
onlineOrder.AddItem(new Item("Мышь", 1500m, 1));

var physicalOrder = new PhysicalOrder(2, "Москва, ул. Пушкина, 10");
physicalOrder.AddItem(new Item("Книга", 900m, 2));
physicalOrder.RemoveItem(physicalOrder.Items.First());

var specializedOrder = new SpecializedOrder(3, "VIP клиент;fragile", 10m);
specializedOrder.AddItem(new Item("Телевизор", 60000m, 1));
specializedOrder.AddItem(new Item("Колонки", 8000m, 1));
specializedOrder.CalculateTotal();

Console.WriteLine();
onlineOrder.DisplayInfo();
Console.WriteLine();
physicalOrder.DisplayInfo();
Console.WriteLine();
specializedOrder.DisplayInfo();


Товар Ноутбук добавлен (доставка: Курьер)
Товар Мышь добавлен (доставка: Курьер)
Товар Книга удалён. Возврат оформлен по адресу: Москва, ул. Пушкина, 10

[Онлайн-заказ] Email: user@example.com
Заказ #1 от 10/6/2025 12:38:27 AM
- Ноутбук x1 = 80000₽
- Мышь x1 = 1500₽
Итого: 81500₽

[Физический заказ] Адрес: Москва, ул. Пушкина, 10
Заказ #2 от 10/6/2025 12:38:27 AM
Итого: 0₽

[Спецзаказ] Условия: VIP клиент;fragile, Скидка: 10%
Заказ #3 от 10/6/2025 12:38:27 AM
- Телевизор x1 = 60000₽
- Колонки x1 = 8000₽
Итого: 64260.00₽
